In [2]:
import numpy as np

class MC(object):
    def __init__(self, rf, credit, times, vol_rf, vol_credit, rho,runs,  antithetic = False):
        self.rf = rf
        self.credit = credit
        self.vol_rf = vol_rf
        self.vol_credit = vol_credit
        self.times = times
        self.runs = runs
        self.anti = antithetic
        self.numfactors = 2
        self.rho = rho

    def pathGen(self,  steps):
        runs = self.runs
        antithetic = self.anti
        np.random.seed(1)
        nsteps = len(steps)
        if antithetic:
            paths = np.ones((2*runs, self.numfactors, nsteps))
        else:
            paths = np.ones((runs, self.numfactors, nsteps))
        paths[:, 0, 0] = self.rf[0]
        paths[:, 1, 0] = self.credit[0]
        for i in range(nsteps)[1:]:
            s0  = np.interp(steps[i-1], self.times, self.credit)
            s1 = np.interp(steps[i], self.times, self.credit)
            rf0  = np.interp(steps[i-1], self.times, self.rf)
            rf1 = np.interp(steps[i], self.times, self.rf)
            driftRf = (rf1 - rf0)
            driftCredit = (s1 - s0)
            deltat = steps[i] - steps[i-1]
            stepvarcov = np.zeros(shape = [self.numfactors, self.numfactors])
            stepvarcov[0,0] = self.vol_rf**2 * deltat
            stepvarcov[1,1] = self.vol_credit**2 * deltat
            stepvarcov[0,1] = self.rho * self.vol_rf * self.vol_credit * deltat
            stepvarcov[1, 0] = stepvarcov[0,1]
            rvars = np.random.multivariate_normal([0]*self.numfactors, stepvarcov, size = runs)
            paths[0:runs, 0, i] = paths[0:runs, 0, i-1]  + driftRf + rvars[:,0]
            paths[0:runs, 1, i] = paths[0:runs, 1, i - 1] + driftCredit + rvars[:, 1]
            if antithetic:
                paths[runs:2*runs, 0, i] = paths[0:runs, 0, i - 1] + driftRf - rvars[:, 0]
                paths[runs:2*runs, 1, i] = paths[0:runs, 1, i - 1] + driftCredit - rvars[:, 1]
        self.paths = paths
        return self.paths

    def price(self, prods, ix):
        if type(prods) != list:
          prodlist = [prods]
        else:
          prodlist = prods
        steps = prodlist[0].steps # we assume same prod steps
        paths = self.pathGen(steps)
        prices = []
        payoffs = []
        for prod in prodlist:
          p = prod.payoff(paths)[ix]
          payoffs.append(p)
          price = 0
          for i, d in enumerate(prod.paydates):
              price += np.mean(p[:,i])
          prices.append(price)
        if type(prods) != list:
          return prices[0]

        return prices

class YieldAutoCallable(object):
    def __init__(self, steps, strikeLow, strikeUp, cpn, fundingSpread, notional = 100):
        self.steps = steps
        self.cpn = cpn
        self.strikeLow = strikeLow
        self.strikeUp = strikeUp
        self.paydates = steps[1:]
        self.fundingSpread = fundingSpread
        self.notional = notional

    def payoff(self, paths):
        nruns, numfactors, nsteps = paths.shape
        steps = self.steps
        out = np.zeros(shape=[nruns, nsteps - 1])
        opt_leg = np.zeros(shape=[nruns, nsteps - 1])
        swp_leg = np.zeros(shape=[nruns, nsteps - 1])
        for j in range(nruns):
            called = False;
            accumulation = 0
            df = 1.
            beginningLibor = paths[j, 0, 0]
            for i in range(nsteps)[1:]:
                libor = paths[j, 0, i]
                creditSpread = paths[j, 1, i]
                y = libor + creditSpread
                deltat = steps[i] - steps[i-1]
                df = df * np.exp(-beginningLibor * deltat)
                if(not called):
                    # first leg
                    out[j,i-1] += (beginningLibor + self.fundingSpread) * deltat * self.notional * df
                    swp_leg[j,i-1] += (beginningLibor + self.fundingSpread) * deltat * self.notional * df
                    beginningLibor = libor

                    #second leg
                    if y > self.strikeUp:
                        accumulation += self.cpn * deltat
                    else:
                        out[j, i - 1] -= (self.cpn * deltat + accumulation)* self.notional * df
                        opt_leg[j, i - 1] -= (self.cpn * deltat + accumulation)* self.notional * df
                        if y < self.strikeLow:
                            called = True
        return out, opt_leg, swp_leg


def priceProduct(rf, credit, times, vol_rf, vol_credit, rho, runs, expiry, frequency, cpn, strikeLow, strikeUp, notional, fundingSpread):
    antithetic = True
    mc = MC(rf, credit, times, vol_rf, vol_credit, rho, runs, antithetic)
    steps = [i * 1. / (12. / frequency) for i in range(int((expiry * 12 / frequency + 1)))]
    cpn_freq = cpn / 12 * frequency
    prod = YieldAutoCallable(steps, strikeLow, strikeUp, cpn_freq, fundingSpread, notional)
    return(mc.price(prod))
    

In [3]:
rf = np.array((0.028, 0.034))
credit = np.array((0.006, 0.01))
times = np.array((0.0, 10.))
vol_rf = 0.005
vol_credit = 0.003
rho = -0.5
runs = 10000

expiry = 7.
frequency = 3.
cpn = 0.045
strikeLow = -0.03
strikeUp = 50.4
notional = 100
fundingSpread = 0.003

In [4]:
mc = MC(rf, credit, times, vol_rf, vol_credit, rho, runs, True)
steps = [i * 1. / (12. / frequency) for i in range(int((expiry * 12 / frequency + 1)))]
cpn_freq = cpn / 12 * frequency
prod = YieldAutoCallable(steps, strikeLow, strikeUp, cpn_freq, fundingSpread, notional)

price = mc.price(prod, 0)

In [5]:
price

np.float64(13.549096348432439)

In [6]:
print("option: ", mc.price(prod, 1))

option:  -7.0967673243143095


In [14]:
print("swap: ", mc.price(prod, 2))

swap:  20.645863672746753


In [8]:
cpn*7

0.315

price

In [9]:
prods = prod

if type(prods) != list:
    prodlist = [prods]
else:
    prodlist = prods
steps = prodlist[0].steps # we assume same prod steps
paths = mc.pathGen(steps)

In [10]:
prod.cpn

0.01125

In [11]:
1.1875*4

4.75

In [12]:
1.125*4

4.5

In [13]:
paths

array([[[ 0.028     ,  0.02387965,  0.02870664, ...,  0.03223857,
          0.03631931,  0.03667004],
        [ 0.006     ,  0.00698071,  0.00501932, ...,  0.00131953,
         -0.00068647, -0.00052669]],

       [[ 0.028     ,  0.02896408,  0.0310673 , ...,  0.03329636,
          0.03651185,  0.0376908 ],
        [ 0.006     ,  0.00436093,  0.00341631, ...,  0.01236642,
          0.01061438,  0.00952736]],

       [[ 0.028     ,  0.02498256,  0.02304724, ...,  0.02963483,
          0.03363586,  0.03683702],
        [ 0.006     ,  0.00431275,  0.00651848, ..., -0.00121422,
         -0.0027199 , -0.00388385]],

       ...,

       [[ 0.028     ,  0.03130429,  0.02154679, ...,  0.03750499,
          0.03340349,  0.03385525],
        [ 0.006     ,  0.00609841,  0.00719432, ..., -0.00278078,
         -0.00379175, -0.0029701 ]],

       [[ 0.028     ,  0.02898555,  0.03050065, ...,  0.03932169,
          0.03806876,  0.0450064 ],
        [ 0.006     ,  0.00258859,  0.00688471, ...,  0.00936